# Tail calls and tail recursion


A function call `f(...)` whose result is immediately returned is a *tail call*:

```python
return f(...)
```

If any operation, no matter how trivial, ever occurs between receiving the result and returning it, then the call is not a tail call:

```python
return 1 + f(...)  # The last operation was +, not the function call.
```

Likewise, even the call in `return f(...)` is not a tail call if that return statement appears in a `try` block that has a `finally` block, or if it appears in a `with` statement. (Please make sure you understand, and can explain, why this follows from the above.)

When a void function calls another void function and then immediately returns, that is also a tail call. Python doesn't have void functions, but if `f` never explicitly returns a value, then `f(...)` always evaluates to `None`, so the function call in

```python
f(...)
return  # Or an implicit return by falling off the end.
```

can be regarded as a tail call in that it always has the same effect as:

```python
return f(...)
```

A *tail-recursive* function is a recursive function whose recursive calls are all tail calls.

A language is said to have *proper tail calls* if, whenever a tail call is made, the caller's stack frame is replaced by the callee's stack frame, so that when $A$ makes a non-tail call to $B$, and $B$ makes a tail call to $C$, $C$ returns directly to $A$. Likewise, if $A$ makes a non-tail call to $B$, and $B$ makes a tail call to $C$, which makes a tail call to $D$, which makes a tail call to $E$, $E$ returns directly to $A$. This holds both when the functions are the same (i.e., in tail recursion) and when they are different. Thus, in languages with proper tail calls, even very "deep" chains of tail calls use only $\mathcal{O}(1)$ call-stack space and don't overflow.

**Python does *not* have proper tail calls**, so functions like `recursion.semifactorial_tail` and `fibonacci.fibonacci_tail` still raise `RecursionError` for large `n`.

Even when using languages without proper tail calls, like Python, recognizing tail recursion is nonetheless of practical use, because it offers insights into how some algorithms work, and it presents opportunities for manual optimization. In particular, **tail recursion can be converted to iteration without requiring a stack**. It's obvious that, for example, semifactorials can be computed iteratively, but in less straightforward situations, especially if some of the recursive calls are tail calls and others aren't, some major optimization opportunities can be hard to identify without tail recursion in mind.

When a unary function that is natural to implement recursively with no helper function is implemented tail-recursively, it is often necessary to use a helper function. The helper function's additional parameters correspond to the non-parameter variables an iterative implementation would introduce to maintain state across iterations.

For example, consider summing the elements of a singly linked list with the iterative accumulator pattern:

```python
def sll_sum(head):
    acc = 0
    while head:
        acc += head.value
        head = head.next
    return acc
```

Implemented tail-recursively with a helper function, the accumulator pattern looks like:

```python
def sll_sum(head):
    def accumulate(acc, node):
        return accumulate(acc + node.value, node.next) if node else acc

    return accumulate(0, head)
```

Contrast this to the non-tail recursive implementation:
    
```python
def sll_sum(head):
    return head.value + sll_sum(head.next) if head else 0
```